In [1]:
#path_name_documents = './Databases/prova/prova4.jsonl'
path_name_documents = './Databases/prova/prova10.jsonl'
#path_name_documents = './Databases/prova/prova50.jsonl'
#path_name_documents = './Databases/prova/prova30000.jsonl'
#path_name_documents = './Databases/prova/prova2000.jsonl'

In [2]:
import time

def time_it(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"Execution time: {end_time - start_time:.5f} seconds")
        return result
    return wrapper

In [3]:
import json
import nltk
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer


stop_words = set(stopwords.words('english'))

def stemmingLemming(filtered_tokens):
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()

    # Perform stemming or lemmatization on filtered tokens
    
    filtered_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    filtered_tokens = [stemmer.stem(token) for token in filtered_tokens]

    return filtered_tokens
    
 
    

def tokenize(path_name):
    
    with open(path_name, "r") as f:
        data = f.readlines()

        # Create an empty list to store the tokenized documents
        tokenized_docs = []

        # Loop through each line in the JSONL file
        for line in data:
            # Parse the JSON string into a Python dictionary
            doc = json.loads(line)

            # Extract the text from the dictionary
            text = doc['text']
            text = text.lower()  # Convert to lowercase
            #text = re.sub(r'\d+', '', text)  # Remove all numbers
            text = text.translate(str.maketrans('', '', string.punctuation))  # Remove all punctuation

            # Tokenize the text using NLTK
            tokens = word_tokenize(text)
            tokensStemLem = stemmingLemming(tokens)

            # Add the tokenized document to the list
            tokenized_docs.append(tokensStemLem)

        # Print the tokenized documents
    return tokenized_docs


tokenized_docs = tokenize(path_name_documents)


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer


def calculateTFIDF(tokenized_docs):
    
    vectorizer = TfidfVectorizer()
    # Fit and transform the tokenized documents into a TF-IDF matrix
    tfidf_matrix = vectorizer.fit_transform([' '.join(doc) for doc in tokenized_docs])

    # Get the feature names (tokens)
    feature_names = vectorizer.get_feature_names_out()

    # Return the TF-IDF matrix and the feature names
    return tfidf_matrix, feature_names,vectorizer
    
        

tfidf_matrix_docs, feature_names_docs,vectorizer  = calculateTFIDF(tokenized_docs)

# DA QUI PARALLEL SPARK

### UNA RIGA

In [5]:
def get_row_values_tfidf(tfidf_matrix, doc_id):
    num_docs = tfidf_matrix.shape[0]
    # Estrai la riga specificata dalla matrice TF-IDF come un array densamente popolato
    row_array = tfidf_matrix.toarray()[doc_id]
    # Creazione della lista di coppie chiave-valore
    row_values = [(index, value) for (index, value) in enumerate(row_array) if value != 0.0]
    # Restituisci la lista di coppie chiave-valore
    return doc_id, row_values,num_docs  


doc_id, row_values, num_docs  = get_row_values_tfidf(tfidf_matrix_docs,0)
#row_values

### TUTTE LE RIGHE

In [6]:
def get_all_row_values_tfidf(tfidf_matrix):
    num_docs = tfidf_matrix.shape[0]
    row_values_list = []
    for doc_id in range(num_docs):
        row_array = tfidf_matrix.toarray()[doc_id]
        row_values = [(index, value) for (index, value) in enumerate(row_array) if value != 0.0]
        row_values_list.append((doc_id, row_values))
    return row_values_list


all_rows_values_list = get_all_row_values_tfidf(tfidf_matrix_docs)
print(all_rows_values_list)

[(0, [(0, 0.0411602263846608), (2, 0.0823204527693216), (3, 0.0823204527693216), (4, 0.0411602263846608), (5, 0.0411602263846608), (6, 0.0411602263846608), (7, 0.0411602263846608), (11, 0.0411602263846608), (12, 0.0411602263846608), (13, 0.0411602263846608), (14, 0.0411602263846608), (15, 0.0411602263846608), (16, 0.0411602263846608), (17, 0.0823204527693216), (18, 0.1234806791539824), (19, 0.0411602263846608), (20, 0.0411602263846608), (21, 0.0823204527693216), (22, 0.0411602263846608), (23, 0.0411602263846608), (24, 0.0411602263846608), (25, 0.0823204527693216), (26, 0.0411602263846608), (27, 0.0411602263846608), (28, 0.0411602263846608), (38, 0.0411602263846608), (39, 0.0411602263846608), (42, 0.0411602263846608), (48, 0.0411602263846608), (59, 0.15217767045579822), (73, 0.0411602263846608), (80, 0.030612074062599096), (81, 0.030612074062599096), (90, 0.03498995282435991), (102, 0.0411602263846608), (103, 0.0411602263846608), (104, 0.0411602263846608), (106, 0.0816489824429282), (12

In [7]:
import os
os.environ['PYSPARK_PYTHON'] = 'C:/Users/lita4/anaconda3/python.exe'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'C:/Users/lita4/anaconda3/python.exe'


In [8]:
import numpy as np


def sim(list1, list2):
    similarity = abs(list1[0] - list2[0])  # Compute similarity as the absolute difference between the values
    return similarity

def b(document):
    # Example function to calculate the threshold 'b' for a document
    threshold = np.mean([value for _, value in document])
    return threshold



In [9]:
# Funzione di mappatura
def my_map(doc_id, document):
    map_results = []
    threshold = b(document)
    for term, value in sorted(document):
        if value > threshold:
            map_results.append((term, (doc_id, document)))
    return map_results

#map_results = my_map(doc_id, document)


In [10]:
def execution_map(num_docs,tfidf_matrix):
    map_results
    for i in range(0,num_docs):
        doc_id, row_values, num_docs  = get_row_values_tfidf(tfidf_matrix_docs,i)
        map_results.append(my_map(doc_id, row_values))
    return map_results



### Funzione funzionante per python

In [11]:
from pyspark.sql import SparkSession
import numpy as np

def my_map_python(doc_id, document):
    map_results = []
    threshold = b(document)
    for term, value in sorted(document):
        if value > threshold:
            map_results.append((term, (doc_id, document)))
    return map_results


def my_reduce_python(map_results, threshold):
    emitted_results = []
    for item in map_results:
        t = item[0]
        doc_id, document = item[1]
        for index, (id1, d1) in enumerate(document):
            for index2 in range(index + 1, len(document)):
                id2, d2 = document[index2]
                if t == max(id1, id2):
                    similarity = sim([d1], [d2])
                    if similarity >= threshold:
                        emitted_results.append((id1, id2, similarity))
    return emitted_results



## Implementazioni per Spark 

In [12]:
#spark = SparkSession.builder.master("local[3]").appName("all-doc-pairs-similarity.com").config("spark.driver.memory", "10g").getOrCreate()

#sc = spark.sparkContext

In [13]:
datas = get_all_row_values_tfidf(tfidf_matrix_docs)


In [32]:
data = [(0, [(2, 0.41137791133379387), (4, 0.6445029922609534), (640, 0.6445029922609534)]),
        (1, [(2, 0.41137791133379387), (3, 0.6445029922609534), (9, 0.6445029922609534)]),
        (2, [(0, 0.5086718718935653), (5, 0.40104274646999605), (6, 0.40104274646999605), (7, 0.5086718718935653), (8, 0.40104274646999605)]),
        (3, [(1, 0.5086718718935653), (4, 0.40104274646999605), (6, 0.40104274646999605), (8, 0.5086718718935653), (12, 0.40104274646999605)]),
        (4, [(3, 0.5086718718935653), (4, 0.40104274646999605), (5, 0.40104274646999605), (7, 0.5086718718935653), (8, 0.40104274646999605)]),
        (5, [(1, 0.5528163151092931), (2, 0.3528554929793508), (5, 0.43584673254990375), (6, 0.43584673254990375), (300, 0.43584673254990375)]),
        (6, [(33, 0.053689068817257726), (53, 0.07218897427142625), (54, 0.12273444662770173), (59, 0.13344860927821803), (65, 0.06136722331385087), (66, 0.07218897427142625), (67, 0.07218897427142625), (75, 0.08573463571936471), (76, 0.06136722331385087), (80, 0.053689068817257726), (86, 0.06136722331385087), (89, 0.10737813763451545), (90, 0.06136722331385087), (94, 0.07218897427142625), (97, 0.07218897427142625), (133, 0.07218897427142625), (135, 0.07218897427142625), (142, 0.06136722331385087), (145, 0.07218897427142625), (147, 0.07218897427142625), (153, 0.10737813763451545), (178, 0.12273444662770173), (190, 0.12860195357904705), (197, 0.06136722331385087), (198, 0.053689068817257726), (204, 0.07218897427142625), (214, 0.07218897427142625), (222, 0.16106720645177316), (237, 0.06136722331385087), (241, 0.07218897427142625), (248, 0.06136722331385087), (259, 0.06136722331385087), (261, 0.06136722331385087), (278, 0.05337944371128721), (281, 0.06136722331385087), (282, 0.07218897427142625), (286, 0.06136722331385087), (287, 0.09546685899760186), (290, 0.06136722331385087), (292, 0.07218897427142625), (299, 0.053689068817257726), (301, 0.07750615818246892), (305, 0.11625923727370337), (312, 0.07218897427142625), (321, 0.053689068817257726), (324, 0.07218897427142625), (329, 0.10737813763451545), (337, 0.053689068817257726), (349, 0.06136722331385087), (354, 0.07218897427142625), (370, 0.07218897427142625), (377, 0.07218897427142625), (378, 0.5053228198999837), (388, 0.34696638412336683), (389, 0.07218897427142625), (393, 0.10556749008926762), (396, 0.06136722331385087), (406, 0.07218897427142625), (407, 0.2147562752690309), (421, 0.07218897427142625), (422, 0.07218897427142625), (423, 0.07218897427142625), (435, 0.1443779485428525), (442, 0.04773342949880093), (446, 0.07218897427142625), (448, 0.06136722331385087), (467, 0.053689068817257726), (487, 0.08573463571936471), (494, 0.042867317859682355), (503, 0.07218897427142625), (513, 0.06136722331385087), (537, 0.06136722331385087), (543, 0.053689068817257726), (545, 0.06136722331385087), (563, 0.02923352404463242), (564, 0.29358694041207967), (569, 0.07750615818246892), (574, 0.16022783451053493), (578, 0.10737813763451545), (605, 0.06136722331385087), (629, 0.03875307909123446), (631, 0.07218897427142625)])]


In [26]:
data2 = [(0, [(0, 0.0411602263846608), (2, 0.0823204527693216), (3, 0.0823204527693216), (4, 0.0411602263846608), (5, 0.0411602263846608), (6, 0.0411602263846608), (7, 0.0411602263846608), (11, 0.0411602263846608), (12, 0.0411602263846608), (13, 0.0411602263846608), (14, 0.0411602263846608), (15, 0.0411602263846608), (16, 0.0411602263846608), (17, 0.0823204527693216), (18, 0.1234806791539824), (19, 0.0411602263846608), (20, 0.0411602263846608), (21, 0.0823204527693216), (22, 0.0411602263846608), (23, 0.0411602263846608), (24, 0.0411602263846608), (25, 0.0823204527693216), (26, 0.0411602263846608), (27, 0.0411602263846608), (28, 0.0411602263846608), (38, 0.0411602263846608), (39, 0.0411602263846608), (42, 0.0411602263846608), (48, 0.0411602263846608), (59, 0.15217767045579822), (73, 0.0411602263846608), (80, 0.030612074062599096), (81, 0.030612074062599096), (90, 0.03498995282435991), (102, 0.0411602263846608), (103, 0.0411602263846608), (104, 0.0411602263846608), (106, 0.0816489824429282), (120, 0.0823204527693216), (123, 0.0823204527693216), (129, 0.024441800502298213), (134, 0.20580113192330401), (136, 0.0411602263846608), (137, 0.1646409055386432), (141, 0.0411602263846608), (146, 0.0411602263846608), (158, 0.0411602263846608), (161, 0.0823204527693216), (163, 0.0411602263846608), (164, 0.0411602263846608), (170, 0.03498995282435991), (173, 0.0411602263846608), (180, 0.03498995282435991), (184, 0.1234806791539824), (197, 0.03498995282435991), (215, 0.0411602263846608), (223, 0.0411602263846608), (230, 0.0411602263846608), (232, 0.0411602263846608), (234, 0.0411602263846608), (236, 0.0411602263846608), (238, 0.0411602263846608), (239, 0.027216327480976066), (242, 0.048883601004596426), (254, 0.0411602263846608), (257, 0.20580113192330401), (263, 0.03498995282435991), (267, 0.03498995282435991), (270, 0.06997990564871982), (273, 0.0411602263846608), (278, 0.060871068182319285), (284, 0.0823204527693216), (285, 0.20993971694615948), (302, 0.03498995282435991), (305, 0.022095971366769235), (307, 0.0411602263846608), (308, 0.0411602263846608), (311, 0.0411602263846608), (319, 0.03498995282435991), (332, 0.0411602263846608), (346, 0.03498995282435991), (347, 0.0411602263846608), (357, 0.09183622218779729), (358, 0.03498995282435991), (359, 0.20993971694615948), (369, 0.0411602263846608), (379, 0.0411602263846608), (385, 0.0411602263846608), (387, 0.030612074062599096), (388, 0.12174213636463857), (395, 0.03498995282435991), (399, 0.024441800502298213), (400, 0.030612074062599096), (415, 0.38488948106795906), (429, 0.34989952824359916), (431, 0.03498995282435991), (433, 0.0823204527693216), (434, 0.03498995282435991), (453, 0.0411602263846608), (458, 0.0411602263846608), (469, 0.0411602263846608), (470, 0.0411602263846608), (471, 0.0411602263846608), (483, 0.0411602263846608), (487, 0.048883601004596426), (491, 0.05443265496195213), (492, 0.0411602263846608), (494, 0.048883601004596426), (501, 0.0411602263846608), (515, 0.0411602263846608), (516, 0.03498995282435991), (520, 0.0411602263846608), (526, 0.0411602263846608), (527, 0.0411602263846608), (551, 0.0411602263846608), (562, 0.06997990564871982), (563, 0.03333635031782873), (564, 0.15217767045579822), (569, 0.022095971366769235), (570, 0.0411602263846608), (571, 0.0823204527693216), (574, 0.05481458082599199), (578, 0.030612074062599096), (589, 0.0411602263846608), (591, 0.0411602263846608), (594, 0.0411602263846608), (597, 0.0411602263846608), (599, 0.0411602263846608), (607, 0.0411602263846608), (615, 0.0816489824429282), (619, 0.34989952824359916), (620, 0.0411602263846608), (624, 0.0411602263846608), (625, 0.0411602263846608), (629, 0.19886374230092313), (632, 0.0411602263846608)]), 
        (1, [(33, 0.053689068817257726), (53, 0.07218897427142625), (54, 0.12273444662770173), (59, 0.13344860927821803), (65, 0.06136722331385087), (66, 0.07218897427142625), (67, 0.07218897427142625), (75, 0.08573463571936471), (76, 0.06136722331385087), (80, 0.053689068817257726), (86, 0.06136722331385087), (89, 0.10737813763451545), (90, 0.06136722331385087), (94, 0.07218897427142625), (97, 0.07218897427142625), (133, 0.07218897427142625), (135, 0.07218897427142625), (142, 0.06136722331385087), (145, 0.07218897427142625), (147, 0.07218897427142625), (153, 0.10737813763451545), (178, 0.12273444662770173), (190, 0.12860195357904705), (197, 0.06136722331385087), (198, 0.053689068817257726), (204, 0.07218897427142625), (214, 0.07218897427142625), (222, 0.16106720645177316), (237, 0.06136722331385087), (241, 0.07218897427142625), (248, 0.06136722331385087), (259, 0.06136722331385087), (261, 0.06136722331385087), (278, 0.05337944371128721), (281, 0.06136722331385087), (282, 0.07218897427142625), (286, 0.06136722331385087), (287, 0.09546685899760186), (290, 0.06136722331385087), (292, 0.07218897427142625), (299, 0.053689068817257726), (301, 0.07750615818246892), (305, 0.11625923727370337), (312, 0.07218897427142625), (321, 0.053689068817257726), (324, 0.07218897427142625), (329, 0.10737813763451545), (337, 0.053689068817257726), (349, 0.06136722331385087), (354, 0.07218897427142625), (370, 0.07218897427142625), (377, 0.07218897427142625), (378, 0.5053228198999837), (388, 0.34696638412336683), (389, 0.07218897427142625), (393, 0.10556749008926762), (396, 0.06136722331385087), (406, 0.07218897427142625), (407, 0.2147562752690309), (421, 0.07218897427142625), (422, 0.07218897427142625), (423, 0.07218897427142625), (435, 0.1443779485428525), (442, 0.04773342949880093), (446, 0.07218897427142625), (448, 0.06136722331385087), (467, 0.053689068817257726), (487, 0.08573463571936471), (494, 0.042867317859682355), (503, 0.07218897427142625), (513, 0.06136722331385087), (537, 0.06136722331385087), (543, 0.053689068817257726), (545, 0.06136722331385087), (563, 0.02923352404463242), (564, 0.29358694041207967), (569, 0.07750615818246892), (574, 0.16022783451053493), (578, 0.10737813763451545), (605, 0.06136722331385087), (629, 0.03875307909123446), (631, 0.07218897427142625)]), 
        (2, [(30, 0.15927444462810794), (31, 0.10618296308540529), (33, 0.039485700343086676), (35, 0.039485700343086676), (51, 0.07021123424408005), (52, 0.04513260974791039), (55, 0.05309148154270264), (59, 0.29443489596159367), (60, 0.05309148154270264), (64, 0.10618296308540529), (72, 0.05309148154270264), (80, 0.039485700343086676), (84, 0.04513260974791039), (96, 0.10618296308540529), (105, 0.05309148154270264), (106, 0.03510561712204002), (107, 0.11845710102926003), (108, 0.05309148154270264), (109, 0.04513260974791039), (115, 0.07897140068617335), (119, 0.05309148154270264), (127, 0.05309148154270264), (131, 0.05309148154270264), (132, 0.05309148154270264), (140, 0.05309148154270264), (149, 0.05309148154270264), (151, 0.04513260974791039), (153, 0.11845710102926003), (175, 0.05309148154270264), (183, 0.05309148154270264), (190, 0.03152682854829443), (193, 0.04513260974791039), (196, 0.05309148154270264), (207, 0.05309148154270264), (216, 0.09026521949582078), (222, 0.039485700343086676), (224, 0.05309148154270264), (227, 0.19742850171543339), (240, 0.05309148154270264), (242, 0.03152682854829443), (246, 0.05309148154270264), (249, 0.05309148154270264), (252, 0.10618296308540529), (268, 0.04513260974791039), (272, 0.05309148154270264), (278, 0.09814496532053121), (281, 0.09026521949582078), (287, 0.03510561712204002), (288, 0.04513260974791039), (289, 0.05309148154270264), (290, 0.04513260974791039), (291, 0.05309148154270264), (294, 0.05309148154270264), (301, 0.17100603553882748), (302, 0.04513260974791039), (316, 0.05309148154270264), (321, 0.039485700343086676), (322, 0.15927444462810794), (323, 0.04513260974791039), (329, 0.1579428013723467), (330, 0.05309148154270264), (331, 0.10618296308540529), (333, 0.039485700343086676), (343, 0.05309148154270264), (345, 0.05309148154270264), (348, 0.10618296308540529), (349, 0.04513260974791039), (352, 0.05309148154270264), (355, 0.039485700343086676), (360, 0.09026521949582078), (363, 0.05309148154270264), (374, 0.05309148154270264), (386, 0.05309148154270264), (388, 0.2748059028974874), (390, 0.05309148154270264), (391, 0.05309148154270264), (393, 0.025879919143470712), (397, 0.04513260974791039), (398, 0.13539782924373117), (399, 0.03152682854829443), (407, 0.039485700343086676), (413, 0.05309148154270264), (414, 0.05309148154270264), (424, 0.05309148154270264), (425, 0.05309148154270264), (449, 0.05309148154270264), (475, 0.04513260974791039), (487, 0.03152682854829443), (488, 0.039485700343086676), (489, 0.07897140068617335), (504, 0.05309148154270264), (514, 0.04513260974791039), (522, 0.3716403707989185), (523, 0.10618296308540529), (525, 0.09026521949582078), (546, 0.15927444462810794), (547, 0.10618296308540529), (548, 0.05309148154270264), (554, 0.05309148154270264), (563, 0.04299967184484812), (564, 0.25517690983338115), (567, 0.05309148154270264), (568, 0.04513260974791039), (574, 0.2121116107815197), (578, 0.039485700343086676), (584, 0.05309148154270264), (604, 0.10618296308540529), (605, 0.09026521949582078), (611, 0.039485700343086676), (613, 0.09026521949582078), (626, 0.05309148154270264), (629, 0.11400402369255166), (630, 0.05309148154270264)]), 
        (3, [(9, 0.13201242899407326), (32, 0.13201242899407326), (33, 0.09818153612139027), (34, 0.11222262529763907), (45, 0.13201242899407326), (56, 0.13201242899407326), (59, 0.1952306455050621), (91, 0.11222262529763907), (98, 0.2640248579881465), (101, 0.11222262529763907), (124, 0.11222262529763907), (126, 0.13201242899407326), (190, 0.31356692969982436), (193, 0.11222262529763907), (210, 0.13201242899407326), (219, 0.3960372869822198), (237, 0.11222262529763907), (256, 0.11222262529763907), (275, 0.11222262529763907), (278, 0.1464229841287966), (287, 0.17458084246109798), (301, 0.14173599648605228), (305, 0.07086799824302614), (310, 0.11222262529763907), (329, 0.19636307224278055), (336, 0.11222262529763907), (338, 0.11222262529763907), (351, 0.13201242899407326), (366, 0.13201242899407326), (384, 0.13201242899407326), (388, 0.1464229841287966), (393, 0.06435064324870728), (419, 0.13201242899407326), (427, 0.11222262529763907), (432, 0.13201242899407326), (454, 0.11222262529763907), (475, 0.11222262529763907), (494, 0.07839173242495609), (498, 0.19636307224278055), (519, 0.13201242899407326), (563, 0.10691905671573201), (564, 0.09761532275253106), (569, 0.07086799824302614), (576, 0.13201242899407326), (627, 0.08729042123054899), (629, 0.07086799824302614)])]

### Functions

In [17]:
def sim(list1, list2):
    # Calcola la similarità come la somma delle differenze normalizzate tra i valori delle due liste
    similarity = sum(abs(x - y) for (x, _), (y, _) in zip(list1, list2))
    max_diff = max(x for x, _ in list1)
    similarity = 1 - similarity / max_diff
    return similarity


# Funzione di mapping
def my_map1(doc_id, document,threshold):
    map_results = []
    threshold = 0.2  # Soglia fissa
    for term, value in sorted(document):
        if value > threshold:
            map_results.append((term, (doc_id, document)))
    return map_results


# Funzione di riduzione
def my_reduce(t, doc_list, threshold):
    print("sono qui")
    emitted_results = []
    for i, doc_item1 in enumerate(doc_list):
        id1, d1 = doc_item1
        for j in range(i + 1, len(doc_list)):
            doc_item2 = doc_list[j]
            id2, d2 = doc_item2
            similarity = sim(d1, d2)
            if similarity >= threshold:
                emitted_results.append((id1, id2, similarity))
    return emitted_results

### Execution with spark

In [18]:
#spark = SparkSession.builder.master("local[3]").appName("all-doc-pairs-similarity.com").config("spark.driver.memory", "5g").getOrCreate()


In [36]:
from pyspark import SparkContext

# Inizializza lo SparkContext
sc = SparkContext()

threshold = 0.2
# Dati di input

# Crea l'RDD di input
input_rdd = sc.parallelize(data2)
#print(input_rdd.collect())

print("--------")
# Esegui la funzione di mapping
mapped_rdd = input_rdd.flatMap(lambda x: my_map1(x[0], x[1],threshold))

# Esegui la funzione di riduzione
reduced_rdd = mapped_rdd.groupByKey().flatMap(lambda x: my_reduce(x[0], list(x[1]),threshold))

# Raccogli i risultati
results = reduced_rdd.collect()

# Elimina duplicati
results = list(set(results))

# Stampa i risultati
for result in results:
    print(result)

# Ferma lo SparkContext
sc.stop()


--------


In [28]:
sc.stop()